# Extracting Data

In [2]:
import pandas as pd
import numpy as np

In [4]:
# --- Load datasets ---
sp500_df = pd.read_csv("SP 500 ESG Risk Ratings.csv")
nifty50_df = pd.read_csv("Nifty50.csv")

In [6]:
# --- Standardize Column Names ---
sp500_df = sp500_df.rename(columns={
    "Name": "Company Name",
    "Total ESG Risk score": "Total_ESG_Risk_Score",
    "Environment Risk Score": "Environment_Score",
    "Governance Risk Score": "Governance_Score",
    "Social Risk Score": "Social_Score",
    "Controversy Level": "Controversy_Level",
    "Controversy Score": "Controversy_Score",
    "ESG Risk Percentile": "ESG_Risk_Percentile",
    "ESG Risk Level": "ESG_Risk_Level"
})

nifty50_df = nifty50_df.rename(columns={
    "company": "Company Name",
    "esg_risk_score_2024": "Total_ESG_Risk_Score",
    "predicted_future_esg_score": "Predicted_ESG_Score",
    "esg_risk_exposure": "ESG_Risk_Exposure",
    "esg_risk_management": "ESG_Risk_Management",
    "esg_risk_level": "ESG_Risk_Level",
    "Controversy Level": "Controversy_Level",
    "controversy_score": "Controversy_Score"
})


In [8]:
# --- Final Set of Columns ---
final_columns = [
    'Symbol', 'Company Name', 'Sector', 'Industry', 'Description',
    'Total_ESG_Risk_Score', 'Predicted_ESG_Score',
    'ESG_Risk_Exposure', 'ESG_Risk_Management', 'ESG_Risk_Level',
    'Environment_Score', 'Governance_Score', 'Social_Score',
    'Controversy_Level', 'Controversy_Score'
]

In [10]:
# --- Value Generator Function ---
def generate_missing_values(df):
    # Only generate if column missing
    if 'Predicted_ESG_Score' not in df.columns:
        df['Predicted_ESG_Score'] = df['Total_ESG_Risk_Score'] + np.random.normal(0, 5, len(df))

    if 'ESG_Risk_Exposure' not in df.columns:
        df['ESG_Risk_Exposure'] = np.random.uniform(30, 70, len(df))

    if 'ESG_Risk_Management' not in df.columns:
        df['ESG_Risk_Management'] = np.random.uniform(30, 70, len(df))

    if 'Environment_Score' not in df.columns:
        df['Environment_Score'] = np.random.uniform(10, 40, len(df))

    if 'Governance_Score' not in df.columns:
        df['Governance_Score'] = np.random.uniform(10, 40, len(df))

    if 'Social_Score' not in df.columns:
        df['Social_Score'] = np.random.uniform(10, 40, len(df))

    if 'Controversy_Score' not in df.columns:
        df['Controversy_Score'] = np.random.randint(1, 101, len(df))

    if 'Controversy_Level' not in df.columns:
        def get_level(score):
            if score <= 20: return 'Low'
            elif score <= 60: return 'Medium'
            else: return 'High'
        df['Controversy_Level'] = [get_level(s) for s in df['Controversy_Score']]

    return df


In [12]:
# --- Generate values for missing columns ---
sp500_df = generate_missing_values(sp500_df)
nifty50_df = generate_missing_values(nifty50_df)


In [14]:
# --- Fill any missing columns to match final schema ---
for col in final_columns:
    if col not in sp500_df.columns:
        sp500_df[col] = np.nan
    if col not in nifty50_df.columns:
        nifty50_df[col] = np.nan


In [16]:
# --- Subset and reorder columns ---
sp500_clean = sp500_df[final_columns]
nifty50_clean = nifty50_df[final_columns]


In [18]:
# --- Combine datasets ---
combined_esg_df = pd.concat([sp500_clean, nifty50_clean], ignore_index=True)


In [20]:
# --- Save cleaned & merged dataset ---
combined_esg_df.to_csv("combined_esg_real_data.csv", index=False)
print("✅ Combined ESG dataset saved as 'combined_esg_real_data.csv'")
print("Combined shape:", combined_esg_df.shape)


✅ Combined ESG dataset saved as 'combined_esg_real_data.csv'
Combined shape: (553, 15)


# Generating Synthetic Data

In [23]:
from faker import Faker
import random

fake = Faker()
num_synthetic = 500

# --- Column Template ---
final_columns = [
    'Symbol', 'Company Name', 'Sector', 'Industry', 'Description',
    'Total_ESG_Risk_Score', 'Predicted_ESG_Score',
    'ESG_Risk_Exposure', 'ESG_Risk_Management', 'ESG_Risk_Level',
    'Environment_Score', 'Governance_Score', 'Social_Score',
    'Controversy_Level', 'Controversy_Score'
]

sectors = ["Technology", "Finance", "Healthcare", "Energy", "Consumer Goods", "Industrials"]
industries = ["Software", "Banks", "Pharmaceuticals", "Oil & Gas", "Retail", "Machinery"]

# --- Generator Function ---
def generate_synthetic_row():
    total_score = round(np.random.uniform(10, 50), 2)
    predicted_score = round(total_score + np.random.normal(0, 3), 2)
    controversy_score = random.randint(1, 100)
    controversy_level = (
        "Low" if controversy_score <= 20 else
        "Medium" if controversy_score <= 60 else "High"
    )
    return {
        'Symbol': fake.lexify(text='???'),
        'Company Name': fake.company(),
        'Sector': random.choice(sectors),
        'Industry': random.choice(industries),
        'Description': fake.catch_phrase(),
        'Total_ESG_Risk_Score': total_score,
        'Predicted_ESG_Score': predicted_score,
        'ESG_Risk_Exposure': round(np.random.uniform(30, 70), 2),
        'ESG_Risk_Management': round(np.random.uniform(30, 70), 2),
        'ESG_Risk_Level': random.choice(['Low', 'Medium', 'High']),
        'Environment_Score': round(np.random.uniform(10, 40), 2),
        'Governance_Score': round(np.random.uniform(10, 40), 2),
        'Social_Score': round(np.random.uniform(10, 40), 2),
        'Controversy_Level': controversy_level,
        'Controversy_Score': controversy_score
    }

# --- Create synthetic dataset ---
synthetic_data = [generate_synthetic_row() for _ in range(num_synthetic)]
synthetic_df = pd.DataFrame(synthetic_data)

# --- Combine with real dataset ---
combined_all_df = pd.concat([combined_esg_df, synthetic_df], ignore_index=True)

# --- Save final dataset ---
combined_all_df.to_csv("final_esg_dataset.csv", index=False)
print("✅ Final ESG dataset with synthetic data saved as 'final_esg_dataset.csv'")
print("Final shape:", combined_all_df.shape)


✅ Final ESG dataset with synthetic data saved as 'final_esg_dataset.csv'
Final shape: (1053, 15)
